In [1]:
import numpy as np
import sysid
import pylab as pl
import time
import pandas as pd
import scipy
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
ss1 = sysid.StateSpaceDiscreteLinear(
    A=0.9, B=0.5, C=1, D=0, Q=0.1, R=0.01, dt=0.1)
ss1

AttributeError: 'float' object has no attribute 'shape'

In [3]:
#np.random.seed(1234)
prbs1 = sysid.prbs(1000)
def f_prbs(t, x, i):
    return prbs1[i]

In [4]:
tf = 10
data = ss1.simulate(f_u=f_prbs, x0=np.matrix(0), tf=tf)
ss1_id = sysid.subspace_det_algo1(y=data.y, u=data.u,
    f=5, p=5, s_tol=1e-1, dt=ss1.dt)
data_id = ss1_id.simulate(f_u=f_prbs, x0=0, tf=tf)
ss1_id

NameError: name 'ss1' is not defined

In [ ]:
pl.plot(data_id.t.T, data_id.x.T, label='id');
pl.plot(data.t.T, data.x.T, label='true');
pl.legend()
pl.grid()
print('fit {:f}%'.format(100*sysid.subspace.nrms(data_id.y, data.y)))

In [ ]:
ss2 = sysid.StateSpaceDiscreteLinear(
    A=np.matrix([[0,0.1,0.2],[0.2,0.3,0.4],[0.4,0.3,0.2]]),
    B=np.matrix([[1,0],[0,1],[0,-1]]),
    C=np.matrix([[1,0,0],[0,1,0]]), D=np.matrix([[0,0],[0,0]]),
    Q=pl.diag([0.1,0.1,0.1]), R=pl.diag([0.04,0.04]), dt=0.1)
ss2

In [ ]:
np.random.seed(1234)
prbs1 = sysid.prbs(1000)
prbs2 = sysid.prbs(1000)
def f_prbs_2d(t, x, i):
    i = i%1000
    return 2*np.matrix([prbs1[i]-0.5, prbs2[i]-0.5]).T

In [ ]:
tf = 10
data2 = ss2.simulate(
    f_u=f_prbs_2d, x0=np.matrix([0,0,0]).T, tf=tf)
ss2_id = sysid.subspace_det_algo1(y=data2.y, u=data2.u,
    f=5, p=5, s_tol=0.2, dt=ss2.dt)
data2_id = ss2_id.simulate(
    f_u=f_prbs_2d,
    x0=np.matrix(pl.zeros(ss2_id.A.shape[0])).T, tf=tf)
ss2_id

In [ ]:
for i in range(2):
    pl.figure(figsize=(15,5))
    pl.plot(data2_id.t.T, data2_id.y[i,:].T,
            label='$y_{:d}$ true'.format(i))
    pl.plot(data2.t.T, data2.y[i,:].T,
            label='$y_{:d}$ id'.format(i))
    pl.legend()
    pl.grid()
print ('fit {:f}%'.format(100*sysid.subspace.nrms(
        data2_id.y, data2.y)))


In [ ]:
# 2 internal states
# ss3 = sysid.StateSpaceDiscreteLinear(
#     A=np.matrix([[0,0.01],[0.2,0.3]]),
#     B=np.matrix([[1,0],[0,1]]),
#     C=np.matrix([[1,0],[0,1]]), 
#     D=np.matrix([[0,0],[0,0]]),
#     Q=pl.diag([0.2,0.1]), # Q == K?
#     R=pl.diag([0.04,0.04]), # R? 
#     dt=0.1)
# 4 internal states
# ss3 = sysid.StateSpaceDiscreteLinear(
#     A=np.matrix([[0,0.01, 0.2, 0.4],[0.1, 0.2, 0.2,0.3],[0.11, 0.12, 0.21,0.13],[0.11, 0.12, 0.21,0.13]]),
#     B=np.matrix([[1,0],[0,1],[1,0],[0,1]]),
#     C=np.matrix([[1,0, 0, 0],[0,1,0,0]]),
#     D=np.matrix([[0,0],[0,0]]),
#     Q=pl.diag([0.2,0.1,0.1,0.1]), # Q == K?
#     R=pl.diag([0.04,0.04]), # R? 
#     dt=0.1)
# ss3
# 4 internal states MIMO [3 IN, 2 OUT]
ss3 = sysid.StateSpaceDiscreteLinear(
    A=np.matrix([[0, 0.01, 0.2, 0.4], [0.1, 0.2, 0.2, 0.3], [0.11, 0.12, 0.21, 0.13], [0.11, 0.12, 0.21, 0.13]]),  # X x X
    B = np.matrix([[1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0]]),  # X x u
    C = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0]]),  # y x x
    D = np.matrix([[0, 0, 0], [0, 0, 0]]),  # y x u
    Q = pl.diag([0.2, 0.1, 0.1, 0.1]),  #  X x X
    R = pl.diag([0.04, 0.04]),  # R? u x u or y x y?
    dt=0.1)

In [ ]:
np.random.seed(1234)
prbs1 = sysid.prbs(1000)
prbs2 = sysid.prbs(1000)
prbs3 = sysid.prbs(1000)
def f_prbs_3d(t, x, i):
    i = i%1000
    return 2*np.matrix([prbs1[i]-0.5, prbs2[i]-0.5, prbs3[i]-0.5]).T

In [ ]:
tf = 10
data3 = ss3.simulate(
    f_u=f_prbs_3d, x0=np.matrix([0,0,0,0]).T, tf=tf)
ss3_id = sysid.subspace_det_algo1(y=data3.y, u=data3.u,
    f=5, p=5, s_tol=0.2, dt=ss3.dt)
data3_id = ss3_id.simulate(
    f_u=f_prbs_3d,
    x0=np.matrix(pl.zeros(ss3_id.A.shape[0])).T, tf=tf)
ss3_id

In [ ]:
print(data3.u.shape)
print(data3.x.shape)
print(data3.y.shape)

In [ ]:
for i in range(2):
    pl.figure(figsize=(15,5))
    pl.plot(data3_id.t.T, data3_id.y[i,:].T,
            label='$y_{:d}$ true'.format(i))
    pl.plot(data3.t.T, data3.y[i,:].T,
            label='$y_{:d}$ id'.format(i))
    pl.legend()
    pl.grid()
print ('fit {:f}%'.format(100*sysid.subspace.nrms(
        data3_id.y, data3.y)))

In [5]:
dt = 1.0
tf = 100

# 4 internal states MIMO [2 IN, 3 OUT]
ss3 = sysid.StateSpaceDiscreteLinear(
    A=np.matrix([[0, 0.01, 0.2, 0.4], [0.1, 0.2, 0.2, 0.3], [0.11, 0.12, 0.21, 0.13], [0.11, 0.12, 0.21, 0.13]]),  # X x X
    B = np.matrix([[1, 0], [0, 1], [1, 0], [0, 1]]),  # X x u
    C = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]]),  # y x x
    D = np.matrix([[0, 0], [0, 1], [0, 0]]),  # y x u
    Q = pl.diag([0.2, 0.1, 0.1, 0.1]),  #  X x X
    R = pl.diag([0.04, 0.04, 0.04]),  # R? y x y
    dt=dt)

np.random.seed(1234)
prbs1 = sysid.prbs(1000)
prbs2 = sysid.prbs(1000)
def f_prbs_3d(t, x, i):
    i = i%1000
    return 2*np.matrix([prbs1[i]-0.5, prbs2[i]-0.5]).T

data3 = ss3.simulate(
    f_u=f_prbs_3d, x0=np.matrix([0,0,0,0]).T, tf=tf)


start_time = time.time()  # Serial
ss3_id = sysid.subspace_det_algo1(y=data3.y, u=data3.u,
    f=5, p=5, s_tol=0.2, dt=ss3.dt)
print("--- Serial:\t\t{} seconds".format(time.time() - start_time))

data3_id = ss3_id.simulate(
    f_u=f_prbs_3d,
    x0=np.matrix(pl.zeros(ss3_id.A.shape[0])).T, tf=tf)

for i in range(3):
    pl.figure(figsize=(15,5))
    pl.plot(data3_id.t.T, data3_id.y[i,:].T,
            label='$y_{:d}$ true'.format(i))
    pl.plot(data3.t.T, data3.y[i,:].T,
            label='$y_{:d}$ id'.format(i))
    pl.legend()
    pl.grid()
print ('fit {:f}%'.format(100*sysid.subspace.nrms(
        data3_id.y, data3.y)))

AttributeError: module 'scipy' has no attribute 'linalg'

In [6]:
tf = 365*8
dt = 1
plot_stuff = False

# TODO: check with randn 500 IN, 3 OUT
data_u = np.random.randn(500, tf)
data_y = np.random.randn(3, tf)

# data_u = data_uu
# data_y = data_yy

print("data_u.shape: {}, data_y.shape: {}".format(data_u.shape, data_y.shape))
print("MIMO [{} IN, {} OUT], {} time-steps.".format(data_u.shape[0], data_y.shape[0], data_u.shape[1]))

def f_prbs_4d(t, x, i):
    return np.matrix(data_u[:,i]).T

start_time = time.time()  # Serial
ss3_id = sysid.subspace_det_algo1(y=data_y, u=data_u,
    f=5,  # 5 Forward steps
    p=5,  # 5 Backward steps                  
    s_tol=0.01, # 0.2
    dt=dt,
    order=-1)
print("--- Serial:\t\t{} seconds".format(time.time() - start_time))

data3_id = ss3_id.simulate(
    f_u=f_prbs_4d,
    x0=np.matrix(pl.zeros(ss3_id.A.shape[0])).T, 
    tf=tf)

if plot_stuff:
    for i in range(3):
        pl.figure(figsize=(15,5))
        pl.plot(data3_id.t.T, data3_id.y[i,:].T,
                label='$y_{:d}$ true'.format(i))
        pl.plot(data3_id.t.T,
                np.matrix(data_y[i,:-1]).T,
                label='$y_{:d}$ id'.format(i))
        pl.legend()
        pl.grid()
print ('fit {:f}%'.format(100*sysid.subspace.nrms(
        data3_id.y, np.array(data_y[:,-1:]))))

data_u.shape: (500, 2920), data_y.shape: (3, 2920)
MIMO [500 IN, 3 OUT], 2920 time-steps.
--- Serial:		55.16209864616394 seconds


TypeError: Cannot convert matrix to cupy.core.core.ndarray

In [ ]:
i = 1
pl.figure(figsize=(15,5))
pl.plot(data3_id.t.T, data3_id.y[i,:].T,
        label='$y_{:d}$ true'.format(i))
pl.plot(data3_id.t.T,
        np.matrix(data_y[i,:-1]).T,
        label='$y_{:d}$ id'.format(i))
pl.legend()
pl.grid()

In [ ]:
df2 = pd.read_csv('sp500_joined_closes.csv')
df2["AAPL"] = df2["AAPL"].astype(float)
df2["MSFT"] = df2["MSFT"].astype(float)
df2["AMD"] = df2["AMD"].astype(float)
df2["INTC"] = df2["INTC"].astype(float)
df2["NVDA"] = df2["NVDA"].astype(float)
df2["AMZN"] = df2["AMZN"].astype(float)
df = df2[["AAPL", "MSFT", "AMD", "INTC", "NVDA", "AMZN"]]
#normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df=(df-df.mean())/(df.std())
data_uu = normalized_df[["AAPL", "MSFT", "AMZN"]].tail(tf).to_numpy().T

data_yy = normalized_df[["AMD", "NVDA", "INTC"]].tail(tf).to_numpy().T
print(data_uu.shape, data_yy.shape)


In [ ]:
import cupy as cp
x_gpu = cp.array([1, 2, 3])
